# ISIC API Encapsulation

In [1]:
import requests


class ISICApi(object):
    def __init__(self, hostname='https://isic-archive.com',
                 username=None, password=None):
        self.baseUrl = f'{hostname}/api/v1'
        self.authToken = None

        if username is not None:
            if password is None:
                password = input(f'Password for user "{username}":')
            self.authToken = self._login(username, password)

    def _makeUrl(self, endpoint):
        return f'{self.baseUrl}/{endpoint}'

    def _login(self, username, password):
        authResponse = requests.get(
            self._makeUrl('user/authentication'),
            auth=(username, password)
        )
        if not authResponse.ok:
            raise Exception(f'Login error: {authResponse.json()["message"]}')

        authToken = authResponse.json()['authToken']['token']
        return authToken

    def get(self, endpoint):
        url = self._makeUrl(endpoint)
        headers = {'Girder-Token': self.authToken} if self.authToken else None
        return requests.get(url, headers=headers)

    def getJson(self, endpoint):
        return self.get(endpoint).json()

    def getJsonList(self, endpoint):
        endpoint += '&' if '?' in endpoint else '?'
        LIMIT = 50
        offset = 0
        while True:
            resp = self.get(
                f'{endpoint}limit={LIMIT:d}&offset={offset:d}'
            ).json()
            if not resp:
                break
            for elem in resp:
                yield elem
            offset += LIMIT

## Connect to API

In [4]:
api = ISICApi(username="spaniernathan", password="4YwB3z&f78BjdBp4W2ghBt^k8YT&HZDS")

## Fetch images

In [6]:
import urllib
import os

savePath = 'ISICArchive/'

if not os.path.exists(savePath):
    os.makedirs(savePath)

imageList = api.getJson('image?limit=20&offset=0&sort=name')

print('Downloading %s images' % len(imageList))
imageDetails = []
for image in imageList:
    print(image['_id'])
    imageFileResp = api.get('image/%s/download' % image['_id'])
    imageFileResp.raise_for_status()
    imageFileOutputPath = os.path.join(savePath, '%s.jpg' % image['name'])
    with open(imageFileOutputPath, 'wb') as imageFileOutputStream:
        for chunk in imageFileResp:
            imageFileOutputStream.write(chunk)

5436e3abbae478396759f0cf
5436e3acbae478396759f0d1
5436e3acbae478396759f0d3
5436e3acbae478396759f0d5
5436e3acbae478396759f0d7
5436e3acbae478396759f0d9
5436e3acbae478396759f0db
5436e3acbae478396759f0dd
5436e3acbae478396759f0df
5436e3acbae478396759f0e1
5436e3acbae478396759f0e3
5436e3acbae478396759f0e5
5436e3acbae478396759f0e7
5436e3adbae478396759f0e9
5436e3adbae478396759f0eb
5436e3adbae478396759f0ed
5436e3adbae478396759f0ef
5436e3adbae478396759f0f1
5436e3adbae478396759f0f3
5436e3adbae478396759f0f5


## Fetch image details & metadata

In [ ]:
print('Fetching metadata for %s images' % len(imageList))
imageDetails = []
for image in imageList:
    print(' ', image['name'])
    # Fetch the full image details
    imageDetail = api.getJson('image/%s' % image['_id'])
    imageDetails.append(imageDetail)
    print('  ', imageDetail['meta']['clinical'])
    print('')
    print('  ', imageDetail)
    print('')

Fetching metadata for 20 images
  ISIC_0000000
   {'age_approx': 55, 'anatom_site_general': 'anterior torso', 'benign_malignant': 'benign', 'diagnosis': 'nevus', 'diagnosis_confirm_type': None, 'melanocytic': True, 'sex': 'female'}

   {'_id': '5436e3abbae478396759f0cf', '_modelType': 'image', 'created': '2014-10-09T19:36:11.989000+00:00', 'creator': {'_id': '5450e996bae47865794e4d0d', 'name': 'User 6VSN'}, 'dataset': {'_accessLevel': -1, '_id': '5a2ecc5e1165975c945942a2', 'description': 'Moles and melanomas.\nBiopsy-confirmed melanocytic lesions. Both malignant and benign lesions are included.', 'license': 'CC-0', 'name': 'UDA-1', 'updated': '2014-11-10T02:39:56.492000+00:00'}, 'meta': {'acquisition': {'image_type': 'dermoscopic', 'pixelsX': 1022, 'pixelsY': 767}, 'clinical': {'age_approx': 55, 'anatom_site_general': 'anterior torso', 'benign_malignant': 'benign', 'diagnosis': 'nevus', 'diagnosis_confirm_type': None, 'melanocytic': True, 'sex': 'female'}}, 'name': 'ISIC_0000000', 'not

## Output metadata in CSV

In [27]:
import csv

outputFileName = 'imagedata'
outputFilePath = 'imagedata/'

if not os.path.exists(outputFilePath):
    os.makedirs(outputFilePath)

metadataFields = set(
    field
    for imageDetail in imageDetails
    for field in imageDetail['meta']['clinical']
)
metadataFields = ['isic_id'] + sorted(metadataFields)

print(metadataFields)

# Write the metadata to a CSV
print('Writing metadata to CSV: %s' % outputFileName+'.csv')

with open(outputFilePath+outputFileName+'.csv', 'w', newline='') as csvFile:
    csvWriter = csv.DictWriter(csvFile, fieldnames=metadataFields, extrasaction='ignore')
    csvWriter.writeheader()
    for imageDetail in imageDetails:
        rowDict = imageDetail['meta']['clinical']
        rowDict['isic_id'] = imageDetail['name']
        csvWriter.writerow(rowDict)

['isic_id', 'age_approx', 'anatom_site_general', 'benign_malignant', 'diagnosis', 'diagnosis_confirm_type', 'melanocytic', 'sex']
Writing metadata to CSV: imagedata.csv


# Create dataset folders

In [9]:
datasetFolder = ["train", "valid", "test"]

for folder in datasetFolder:
    if not os.path.exists('imagedata/'+folder):
        os.makedirs('imagedata/'+folder)

# Model training

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_path = 'imagedata/train'
valid_path = 'imagedata/valid'
test_path = 'imagedata/test'
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['benign', 'malignant'], batch_size=10)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['benign', 'malignant'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['benign', 'malignant'], batch_size=10, shuffle=False)


imgs, labels = next(train_batches)

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dropout(0.4),
    Dense(units=2, activation='softmax')
])

model.summary()

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=10,
    verbose=1
)

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [ ]:
#np.round(predictions)
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
#test_batches.classes

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['benign','malignant']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')